In [141]:
import pandas as pd
import numpy as np
# np.random.seed(666)
from scipy.sparse import coo_array, csr_array,csr_matrix
import tSB
import pickle

file_name = "tracking/qubo_ev1004_1p0.pickle"
objects = []

with (open(file_name, "rb")) as f:
    while True:
        try:
            objects.append(pickle.load(f))
        except EOFError:
            break
#print(objects)
qubo = objects

index_mapping = {}
custom_index = 0

# Iterate over the QUBO dictionary and create the index mapping
for pair in qubo[0]:
    for index in pair:
        if index not in index_mapping:
            index_mapping[index] = custom_index
            #print('index', custom_index, 'pair', pair[0])
            custom_index += 1


row = []
col = []
data = []
vector = np.zeros(custom_index)
for pair, value in qubo[0].items():
    index1 = index_mapping[pair[0]]
    index2 = index_mapping[pair[1]]
    if index1 == index2:
        vector[index1] = value
    else:
        row.append(index1)
        col.append(index2)
        data.append(value)

data = np.array(data)
row = np.array(row)
col = np.array(col)
matrix_ori = coo_array((data, (row, col)), shape=(custom_index, custom_index))
matrix = (matrix_ori + matrix_ori.T) / 2
# matrix_ori


################ building model ######################

J = matrix / 4
h = vector / 2 + matrix.sum(axis=0) / 2
offset = matrix.sum() / 4 + vector.sum()/2
J = 2 * J
h =  h[:, np.newaxis]
scale_factor = np.max([np.abs(J).max(), np.abs(h).max()])

J.shape,h.shape,type(J),scale_factor


((109498, 109498),
 (109498, 1),
 scipy.sparse._arrays.csr_array,
 50.75927752871023)

In [ ]:
# solving Ising problem
s = tSB.SB(A=-J,h=-h,tabu=None,xi=None,n_iter=4000,batch_size=1,dt=1.,num_tabu=2)
s.update_b()


In [146]:
0.5 * np.sum(J.dot(np.sign(s.x)) * np.sign(s.x), axis=0)+h[:,0].T.dot(np.sign(s.x))

array([-449203.03966766])

In [ ]:
s1 = tSB.SB(A=-J,h=-h,tabu=None,xi=None,n_iter=300,batch_size=10,dt=1.,num_tabu=2)
s1.update_d()
tabu_sample = np.sign(s1.x).copy()
s1.xi

1.0069431652124656

In [163]:
s1 = tSB.SB(A=-J,h=-h,tabu=0.01*tabu_sample,xi=1,n_iter=2700,batch_size=1,dt=1.,num_tabu=2)
s1.update_d()


In [164]:
sample = np.sign(s.x).copy()
energy = (0.5 * np.sum(J.dot(sample) * sample, axis=0)+h.T.dot(sample))[0,:]
opt_index = np.argmin(energy)
opt_energy = energy[opt_index]
opt_sample = sample[:, opt_index]

sample1 = np.sign(s1.x).copy()
energy1 = (0.5 * np.sum(J.dot(sample1) * sample1, axis=0)+h.T.dot(sample1))[0,:]
opt_index1 = np.argmin(energy1)
opt_energy1 = energy1[opt_index1]
opt_sample1 = sample1[:, opt_index1]


print(f'bSB minimum energy:{opt_energy}')
print(energy.mean())
# print(f'optimal solution:{opt_sample}')
print(f'TEbSB minimum energy:{opt_energy1}')
print(energy1.mean())

bSB minimum energy:-449203.03966765874
-449203.03966765874
TEbSB minimum energy:-449356.5395088129
-449356.5395088129


(1, 10)